# DraftGap



notebook used to create this blog post:
https://loldraftai.com/blog/draftgap-vs-loldraftai-comparison

In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

## API Endpoint Configuration


In [ ]:
# Configure the API endpoint
API_URL = "http://localhost:3000/analyze"

# Check if the server is running
try:
    health_response = requests.get("http://localhost:3000/health")
    health_data = health_response.json()
    print(f"Server status: {health_data['status']}")
    print(f"Datasets loaded: {health_data['datasetsLoaded']}")
except requests.exceptions.ConnectionError:
    print(
        "⚠️ API server not running! Please start the server with 'npm start' in the draftgap-api directory."
    )

In [ ]:
# Creating a CSV with 1000 rows comparing both models
import os
import glob
import pickle
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

# Load label encoders to decode champion IDs
ENCODERS_PATH = os.path.join('/Users/loyd/draftking/apps/machine-learning/data', "label_encoders.pkl")
with open(ENCODERS_PATH, "rb") as f:
    label_encoders = pickle.load(f)

# Function to fetch DraftGap prediction
def fetch_draftgap_prediction(champion_ids):
    # Split the champion IDs into ally (blue) and enemy (red) teams
    ally_team = {
        "0": int(champion_ids[0]),  # Top
        "1": int(champion_ids[1]),  # Jungle
        "2": int(champion_ids[2]),  # Mid
        "3": int(champion_ids[3]),  # ADC
        "4": int(champion_ids[4]),  # Support
    }
    
    enemy_team = {
        "0": int(champion_ids[5]),  # Top
        "1": int(champion_ids[6]),  # Jungle
        "2": int(champion_ids[7]),  # Mid
        "3": int(champion_ids[8]),  # ADC
        "4": int(champion_ids[9]),  # Support
    }
    
    payload = {
        "allyTeam": ally_team,
        "enemyTeam": enemy_team,
        "riskLevel": "medium"
    }
    
    try:
        response = requests.post("http://localhost:3000/analyze", json=payload)
        if response.status_code == 200:
            result = response.json()
            return result['winrate']
        else:
            print(f"Error from DraftGap API: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None

# Function to fetch LoLDraftAI prediction
def fetch_loldraftai_prediction(champion_ids):
    input_data = {
        "champion_ids": champion_ids.tolist(),
        "numerical_elo": 3,  # Emerald
        "patch": "15.04",
    }
    
    headers = {"X-API-Key": "example_token"}
    
    try:
        response = requests.post(
            "http://localhost:8000/predict", json=input_data, headers=headers
        )
        if response.status_code == 200:
            return response.json()["win_probability"]
        else:
            print(f"Error from LoLDraftAI API: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None

# Load and filter test data
def load_filtered_test_data(max_rows=1000):
    all_files = sorted(glob.glob("/Users/loyd/draftking/apps/machine-learning/data/prepared_data/test/test_*.parquet"))
    
    filtered_data = []
    total_rows = 0
    
    for file_path in all_files:
        if total_rows >= max_rows:
            break
            
        df = pd.read_parquet(file_path)
        # Since there are no games with version 15.4, let's use version 15.3 instead
        filtered_df = df[(df['gameVersionMajorPatch'] == 15) & (df['gameVersionMinorPatch'] == 3)]
        
        if len(filtered_df) > 0:
            # Take only what we need to reach max_rows
            rows_needed = min(max_rows - total_rows, len(filtered_df))
            filtered_data.append(filtered_df.head(rows_needed))
            total_rows += rows_needed
            print(f"Added {rows_needed} rows from {file_path}")
            
    if total_rows < max_rows:
        print(f"Could only find {total_rows} rows with version 15.3")
        
    # Combine all filtered data
    if filtered_data:
        return pd.concat(filtered_data, ignore_index=True)
    else:
        return pd.DataFrame()

# Main processing function
def create_validation_predictions_csv():
    # Load and filter data to get 1000 rows
    df = load_filtered_test_data(max_rows=5000)  # Using 100 for a quick demonstration
    
    if len(df) == 0:
        print("No data found matching criteria")
        return None
    
    print(f"Processing {len(df)} matches from version 15.3")
    
    # Initialize the results dataframe
    results = []
    
    # For each row in the filtered dataframe
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing matches"):
        # Get the encoded champion IDs and convert to original IDs
        encoded_champion_ids = row['champion_ids']
        original_champion_ids = label_encoders['champion_ids'].inverse_transform(encoded_champion_ids)
        
        # Get match ID
        match_id = row['matchId']
        
        # Get actual result (True = blue win, False = red win)
        actual_result = int(row['win_prediction'])
        
        # Get predictions from both models
        draftgap_prediction = fetch_draftgap_prediction(original_champion_ids)
        loldraftai_prediction = fetch_loldraftai_prediction(np.array(original_champion_ids))
        
        # Store the results
        results.append({
            'matchId': match_id,
            'draftgap_prediction': draftgap_prediction if draftgap_prediction is not None else float('nan'),
            'loldraftai_prediction': loldraftai_prediction if loldraftai_prediction is not None else float('nan'),
            'actual_result': actual_result,
            'champion_ids': original_champion_ids.tolist()  # Store the original champion IDs
        })
    
    # Create dataframe from results
    results_df = pd.DataFrame(results)
    
    # Save as CSV
    results_df.to_csv('validation_predictions.csv', index=False)
    
    print(f"Saved {len(results_df)} predictions to validation_predictions.csv")
    
    return results_df

# Run the main function to generate the results
results_df = create_validation_predictions_csv()

In [ ]:
# Analyze the results - focusing only on accuracy and model comparison
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix

# Function to analyze and compare models
def analyze_model_comparison(df):
    # Make sure we only use rows where both predictions are available
    valid_df = df.dropna()
    
    if len(valid_df) == 0:
        print("No valid predictions to analyze")
        return
    
    # Convert predictions to binary outcomes (>0.5 = blue win)
    draftgap_binary = (valid_df['draftgap_prediction'] > 0.5).astype(int)
    loldraftai_binary = (valid_df['loldraftai_prediction'] > 0.5).astype(int)
    
    # Calculate accuracy for each model
    draftgap_accuracy = accuracy_score(valid_df['actual_result'], draftgap_binary)
    loldraftai_accuracy = accuracy_score(valid_df['actual_result'], loldraftai_binary)
    
    print(f"Total valid samples: {len(valid_df)}")
    print("\nOverall Accuracy:")
    print(f"- DraftGap: {draftgap_accuracy:.4f} ({int(draftgap_accuracy * len(valid_df))}/{len(valid_df)} correct)")
    print(f"- LoLDraftAI: {loldraftai_accuracy:.4f} ({int(loldraftai_accuracy * len(valid_df))}/{len(valid_df)} correct)")
    
    # Check when models agree vs disagree
    agree_mask = draftgap_binary == loldraftai_binary
    agree_count = agree_mask.sum()
    disagree_count = len(valid_df) - agree_count
    
    print(f"\nModel Agreement: {agree_count}/{len(valid_df)} samples ({agree_count/len(valid_df):.2%})")
    print(f"Model Disagreement: {disagree_count}/{len(valid_df)} samples ({disagree_count/len(valid_df):.2%})")
    
    # Compute accuracy when models agree
    if agree_count > 0:
        agree_df = valid_df[agree_mask]
        agree_accurate = (draftgap_binary[agree_mask] == agree_df['actual_result']).mean()
        print(f"Accuracy when models agree: {agree_accurate:.4f}")
    
    # Compute accuracy when models disagree
    if disagree_count > 0:
        disagree_df = valid_df[~agree_mask]
        draftgap_disagree_acc = accuracy_score(disagree_df['actual_result'], draftgap_binary[~agree_mask])
        loldraftai_disagree_acc = accuracy_score(disagree_df['actual_result'], loldraftai_binary[~agree_mask])
        print(f"When models disagree, DraftGap accuracy: {draftgap_disagree_acc:.4f}")
        print(f"When models disagree, LoLDraftAI accuracy: {loldraftai_disagree_acc:.4f}")
    
    # Create confusion matrices
    draftgap_cm = confusion_matrix(valid_df['actual_result'], draftgap_binary)
    loldraftai_cm = confusion_matrix(valid_df['actual_result'], loldraftai_binary)
    
    # Plot confusion matrices
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    sns.heatmap(draftgap_cm, annot=True, fmt='d', cmap='Blues', 
               xticklabels=['Red Win', 'Blue Win'], yticklabels=['Red Win', 'Blue Win'], ax=axes[0])
    axes[0].set_title('DraftGap Confusion Matrix')
    axes[0].set_ylabel('Actual')
    axes[0].set_xlabel('Predicted')
    
    sns.heatmap(loldraftai_cm, annot=True, fmt='d', cmap='Blues', 
               xticklabels=['Red Win', 'Blue Win'], yticklabels=['Red Win', 'Blue Win'], ax=axes[1])
    axes[1].set_title('LoLDraftAI Confusion Matrix')
    axes[1].set_ylabel('Actual')
    axes[1].set_xlabel('Predicted')
    
    plt.tight_layout()
    plt.show()
    
    # Create direct comparison scatter plot
    plt.figure(figsize=(10, 8))
    
    # We'll color points by their actual outcome
    scatter = plt.scatter(valid_df['draftgap_prediction'], 
                         valid_df['loldraftai_prediction'], 
                         c=valid_df['actual_result'], 
                         cmap='coolwarm', 
                         alpha=0.7, 
                         s=50)
    
    plt.axhline(y=0.5, color='gray', linestyle='--')
    plt.axvline(x=0.5, color='gray', linestyle='--')
    plt.grid(True, alpha=0.3)
    plt.xlabel('DraftGap Prediction (Blue Win Probability)')
    plt.ylabel('LoLDraftAI Prediction (Blue Win Probability)')
    plt.title('Comparison of Model Predictions')
    plt.colorbar(scatter, label='Actual Outcome (0=Red Win, 1=Blue Win)')
    
    # Add regions
    plt.text(0.25, 0.75, "DraftGap: Red Win\nLoLDraftAI: Blue Win", 
             horizontalalignment='center', size=10, alpha=0.7)
    plt.text(0.75, 0.25, "DraftGap: Blue Win\nLoLDraftAI: Red Win", 
             horizontalalignment='center', size=10, alpha=0.7)
    plt.text(0.25, 0.25, "Both Predict\nRed Win", 
             horizontalalignment='center', size=10, alpha=0.7)
    plt.text(0.75, 0.75, "Both Predict\nBlue Win", 
             horizontalalignment='center', size=10, alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    # Return metrics for further use
    return {
        'draftgap_accuracy': draftgap_accuracy,
        'loldraftai_accuracy': loldraftai_accuracy,
        'agreement_rate': agree_count/len(valid_df),
        'draftgap_cm': draftgap_cm,
        'loldraftai_cm': loldraftai_cm
    }

# If results_df exists, analyze it
if 'results_df' in locals() and results_df is not None and len(results_df) > 0:
    metrics = analyze_model_comparison(results_df)
else:
    print("No results available - run the data generation cell first")

In [ ]:
# Summary of model comparison with detailed champion information
import pandas as pd
import numpy as np
from utils.rl.champions import Champion

# If metrics exists, create a summary
if 'metrics' in locals() and metrics is not None and 'results_df' in locals() and results_df is not None:
    # Calculate absolute error for each prediction
    valid_df = results_df.dropna().copy()
    
    valid_df['draftgap_error'] = np.abs(valid_df['draftgap_prediction'] - valid_df['actual_result'])
    valid_df['loldraftai_error'] = np.abs(valid_df['loldraftai_prediction'] - valid_df['actual_result'])
    
    # Determine which model has closer prediction for each row
    valid_df['better_model'] = np.where(
        valid_df['draftgap_error'] < valid_df['loldraftai_error'],
        'DraftGap',
        np.where(
            valid_df['loldraftai_error'] < valid_df['draftgap_error'],
            'LoLDraftAI',
            'Tie'
        )
    )
    
    # Calculate mean absolute error
    draftgap_mae = valid_df['draftgap_error'].mean()
    loldraftai_mae = valid_df['loldraftai_error'].mean()
    
    # Count how many times each model is better
    model_comparison = valid_df['better_model'].value_counts()
    
    print("# Model Comparison Summary\n")
    
    # Print overall accuracy comparison
    print("## Overall Accuracy")
    if metrics['draftgap_accuracy'] > metrics['loldraftai_accuracy']:
        acc_diff = metrics['draftgap_accuracy'] - metrics['loldraftai_accuracy']
        winner = "DraftGap"
    elif metrics['loldraftai_accuracy'] > metrics['draftgap_accuracy']:
        acc_diff = metrics['loldraftai_accuracy'] - metrics['draftgap_accuracy']
        winner = "LoLDraftAI"
    else:
        acc_diff = 0
        winner = "Tie"
        
    print(f"Winner: {winner}" + (f" (+{acc_diff:.4f})" if acc_diff > 0 else ""))
    
    # Print error comparison
    print("\n## Mean Absolute Error")
    if draftgap_mae < loldraftai_mae:
        error_diff = loldraftai_mae - draftgap_mae
        error_winner = "DraftGap"
    elif loldraftai_mae < draftgap_mae:
        error_diff = draftgap_mae - loldraftai_mae
        error_winner = "LoLDraftAI"
    else:
        error_diff = 0
        error_winner = "Tie"
        
    print(f"DraftGap MAE: {draftgap_mae:.4f}")
    print(f"LoLDraftAI MAE: {loldraftai_mae:.4f}")
    print(f"Winner: {error_winner}" + (f" (-{error_diff:.4f})" if error_diff > 0 else ""))
    
    # Print how often each model is closer to the actual result
    print("\n## Row-by-Row Comparison")
    for model, count in model_comparison.items():
        percentage = count / len(valid_df) * 100
        print(f"{model}: {count} rows ({percentage:.1f}%)")
    
    # Create a map from champion ID to name
    champion_id_to_name = {c.id: c.display_name for c in Champion}
    
    # OPTIMIZATION: The champion IDs are already in create_validation_predictions_csv()
    # We don't need to reload the whole dataset for each match, just append champion data
    
    # First, modify our results_df to include champion IDs
    # We need to merge it with the detailed data structure
    detailed_rows = []
    
    for idx, row in valid_df.iterrows():
        match_id = row['matchId']
        
        # Create a dictionary for this row with all the comparison data
        detailed_row = {
            'matchId': match_id,
            'draftgap_prediction': row['draftgap_prediction'],
            'loldraftai_prediction': row['loldraftai_prediction'],
            'actual_result': row['actual_result'],
            'draftgap_error': row['draftgap_error'],
            'loldraftai_error': row['loldraftai_error'],
            'better_model': row['better_model']
        }
        
        # Add champion information directly from the results_df
        # The champion_ids should be stored in the original results_df from create_validation_predictions_csv
        if 'champion_ids' in row:
            original_champion_ids = row['champion_ids']
        else:
            # If not available, use dummy values
            original_champion_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            
        detailed_row['champion_ids'] = original_champion_ids
        
        # Add position-specific champion names
        positions = ['Top', 'Jungle', 'Mid', 'ADC', 'Support']
        
        for i, position in enumerate(positions):
            # Blue team (positions 0-4)
            champ_id = int(original_champion_ids[i]) if i < len(original_champion_ids) else 0
            detailed_row[f'Blue_{position}'] = champion_id_to_name.get(champ_id, f"Unknown ({champ_id})")
            
            # Red team (positions 5-9)
            champ_id = int(original_champion_ids[i+5]) if i+5 < len(original_champion_ids) else 0
            detailed_row[f'Red_{position}'] = champion_id_to_name.get(champ_id, f"Unknown ({champ_id})")
        
        detailed_rows.append(detailed_row)
    
    # Create the detailed dataframe
    comparison_df = pd.DataFrame(detailed_rows)
    
    # Save the enhanced comparison CSV
    comparison_df.to_csv('model_comparison_details.csv', index=False)
    print(f"\nEnhanced detailed comparison saved to model_comparison_details.csv")
    print(f"Added champion IDs and names for each position for easier human readability")
    
else:
    print("No metrics available - run the analysis cell first")

In [ ]:
# Let's preview the enhanced comparison CSV
if 'comparison_df' in locals() and comparison_df is not None and len(comparison_df) > 0:
    # Display a sample of the enhanced comparison data
    print("Sample of the enhanced model comparison details:")
    display_columns = [
        'matchId', 'actual_result', 
        'draftgap_prediction', 'loldraftai_prediction', 
        'better_model',
        'Blue_Top', 'Blue_Jungle', 'Blue_Mid', 'Blue_ADC', 'Blue_Support',
        'Red_Top', 'Red_Jungle', 'Red_Mid', 'Red_ADC', 'Red_Support'
    ]
    
    # Display the first few rows with just the key columns
    print(comparison_df[display_columns].head())
    
    # Show the column names in the CSV
    print("\nAll columns in the model_comparison_details.csv:")
    print(", ".join(comparison_df.columns))
    
    # Show some statistics
    print(f"\nTotal matches analyzed: {len(comparison_df)}")
    print(f"DraftGap better: {len(comparison_df[comparison_df['better_model'] == 'DraftGap'])} matches")
    print(f"LoLDraftAI better: {len(comparison_df[comparison_df['better_model'] == 'LoLDraftAI'])} matches")
    print(f"Tie: {len(comparison_df[comparison_df['better_model'] == 'Tie'])} matches")
else:
    print("No comparison data available - run the analysis cells first")